In [1]:
import pandas as pd
from datetime import datetime, timedelta
from prophet import Prophet

df = pd.read_csv('log-info.csv')

df.rename(columns={'timestamp' : 'ds'}, inplace=True)
df

,userid,sessionid,ds,action,value,category,age,gender,city,point
0,user_10000,1,2021-12-28T02:33:15,mainpage,0.000000,None,48,М,Омск,"[54.989347,73.368221]"
1,user_10000,1,2021-12-28T02:33:29,search,0.000000,Продовольственные товары,48,М,Омск,"[54.989347,73.368221]"
2,user_10000,1,2021-12-28T02:33:47,product,0.000000,Продовольственные товары,48,М,Омск,"[54.989347,73.368221]"
3,user_10000,1,2021-12-28T02:33:59,mainpage,0.000000,None,48,М,Омск,"[54.989347,73.368221]"
4,user_10000,1,2021-12-28T02:34:10,category,0.000000,Товары для детей,48,М,Омск,"[54.989347,73.368221]"
...,...,...,...,...,...,...,...,...,...,...
14553412,user_14999,529,2025-01-02T04:00:01,cart,23613.725533,Продовольственные товары,34,Ж,Челябинск,"[55.159902,61.402554]"
14553413,user_14999,529,2025-01-02T04:00:45,checkout,992.000000,None,34,Ж,Челябинск,"[55.159902,61.402554]"
14553414,user_14999,529,2025-01-02T04:01:08,confirmation,992.000000,None,34,Ж,Челябинск,"[55.159902,61.402554]"
14553415,user_14999,530,2025-01-03T04:01:15,search,0.000000,Продовольственные товары,34,Ж,Челябинск,"[55.159902,61.402554]"


In [2]:
predictions = 365

def make_forecast(data):
    m = Prophet()
    m.fit(data)
    future = m.make_future_dataframe(periods=predictions)
    return m.predict(future)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']]

In [3]:
dau = df[['ds', 'userid']].copy()

dau['ds'] = dau['ds'].apply(lambda x: x[:10])


dau = dau.groupby('ds')['userid'].apply(lambda x: len(x.unique())).reset_index() \
    .rename(columns={'userid' : 'y'})

dau = make_forecast(dau).iloc[-predictions:] \
    .rename(columns={'yhat_lower' : 'dau_lower', 'yhat_upper' : 'dau_upper', 'yhat' : 'dau'})
dau

18:48:46 - cmdstanpy - INFO - Chain [1] start processing
18:48:47 - cmdstanpy - INFO - Chain [1] done processing


,ds,dau_lower,dau_upper,dau
1105,2025-01-04,1932.439354,2303.164829,2119.630807
1106,2025-01-05,1921.717892,2285.614041,2103.358113
1107,2025-01-06,1951.723196,2317.053495,2122.732723
1108,2025-01-07,1920.784499,2291.814987,2105.918379
1109,2025-01-08,1959.786915,2319.479967,2135.122837
...,...,...,...,...
1465,2025-12-30,1866.146176,2250.088061,2065.888538
1466,2025-12-31,1909.851557,2289.405250,2092.868413
1467,2026-01-01,1912.319482,2290.043067,2091.352384
1468,2026-01-02,1908.980605,2299.680062,2102.887560


In [4]:
revenue = df.loc[df['action'] == 'confirmation'][['ds', 'value']]

revenue['ds'] = revenue['ds'].apply(lambda x: x[:10])


revenue = revenue.groupby('ds')['value'].sum().reset_index() \
    .rename(columns={'value' : 'y'})

revenue = make_forecast(revenue).iloc[-predictions:] \
    .rename(columns={'yhat_lower' : 'revenue_lower', 'yhat_upper' : 'revenue_upper', 'yhat' : 'revenue'})
revenue

18:48:50 - cmdstanpy - INFO - Chain [1] start processing
18:48:51 - cmdstanpy - INFO - Chain [1] done processing


,ds,revenue_lower,revenue_upper,revenue
1104,2025-01-04,586459.903557,863492.143353,720781.799374
1105,2025-01-05,590721.293507,865274.987477,728055.696018
1106,2025-01-06,573324.219023,854438.395332,715677.851171
1107,2025-01-07,561100.892940,854228.820819,711372.336071
1108,2025-01-08,563744.555439,829593.944923,698388.099004
...,...,...,...,...
1464,2025-12-30,583551.033722,865470.746605,728195.458086
1465,2025-12-31,576605.008336,853017.475937,716789.589539
1466,2026-01-01,566477.423852,846346.823214,708208.194639
1467,2026-01-02,593490.704077,878390.661027,734369.453086


In [5]:
orders = df.loc[df['action'] == 'confirmation'][['ds', 'value']]

orders['ds'] = orders['ds'].apply(lambda x: x[:10])


orders = orders.groupby('ds')['value'].count().reset_index() \
    .rename(columns={'value' : 'y'})

orders = make_forecast(orders).iloc[-predictions:] \
    .rename(columns={'yhat_lower' : 'orders_lower', 'yhat_upper' : 'orders_upper', 'yhat' : 'orders'})
orders

18:48:54 - cmdstanpy - INFO - Chain [1] start processing
18:48:54 - cmdstanpy - INFO - Chain [1] done processing


,ds,orders_lower,orders_upper,orders
1104,2025-01-04,85.666377,117.161781,102.028342
1105,2025-01-05,85.639062,116.317008,100.905671
1106,2025-01-06,85.509349,116.126822,100.667862
1107,2025-01-07,83.144586,115.247611,100.409916
1108,2025-01-08,84.498328,115.582309,100.953102
...,...,...,...,...
1464,2025-12-30,84.312953,116.682396,100.342103
1465,2025-12-31,85.535046,117.193864,101.015296
1466,2026-01-01,83.671094,113.396037,99.417269
1467,2026-01-02,85.641095,117.969495,101.893977


In [7]:
result = dau.merge(revenue, how='outer', on='ds') \
    .merge(orders, how='outer', on='ds')

result.rename(columns={'ds' : 'date'}, inplace=True)

result.to_csv('forecast.csv', index=False)
result

,date,dau_lower,dau_upper,dau,revenue_lower,revenue_upper,revenue,orders_lower,orders_upper,orders
0,2025-01-04,1932.439354,2303.164829,2119.630807,586459.903557,863492.143353,720781.799374,85.666377,117.161781,102.028342
1,2025-01-05,1921.717892,2285.614041,2103.358113,590721.293507,865274.987477,728055.696018,85.639062,116.317008,100.905671
2,2025-01-06,1951.723196,2317.053495,2122.732723,573324.219023,854438.395332,715677.851171,85.509349,116.126822,100.667862
3,2025-01-07,1920.784499,2291.814987,2105.918379,561100.892940,854228.820819,711372.336071,83.144586,115.247611,100.409916
4,2025-01-08,1959.786915,2319.479967,2135.122837,563744.555439,829593.944923,698388.099004,84.498328,115.582309,100.953102
...,...,...,...,...,...,...,...,...,...,...
360,2025-12-30,1866.146176,2250.088061,2065.888538,583551.033722,865470.746605,728195.458086,84.312953,116.682396,100.342103
361,2025-12-31,1909.851557,2289.405250,2092.868413,576605.008336,853017.475937,716789.589539,85.535046,117.193864,101.015296
362,2026-01-01,1912.319482,2290.043067,2091.352384,566477.423852,846346.823214,708208.194639,83.671094,113.396037,99.417269
363,2026-01-02,1908.980605,2299.680062,2102.887560,593490.704077,878390.661027,734369.453086,85.641095,117.969495,101.893977
